### Cargamos herramientas necesarias

In [729]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from funciones_Brasil import porcentaje_nulos
from pandas import ExcelWriter
import itertools

pd.set_option('display.max_columns', 200) 

df_2013 = pd.read_csv('../datos/datos-2013.csv', index_col = 0, encoding='ISO-8859-1', sep=';')
df_2014 = pd.read_csv('../datos/datos-2014.csv', index_col = 0, encoding='ISO-8859-1', sep=';')
df_2015 = pd.read_csv('../datos/datos-2015.csv', index_col = 0, encoding='ISO-8859-1', sep=';')
df_2016 = pd.read_csv('../datos/datos-2016.csv', index_col = 0, encoding='ISO-8859-1', sep=';')
df_2017 = pd.read_csv('../datos/datos-2017.csv', index_col = 0, encoding='ISO-8859-1', sep=';')
df_2018 = pd.read_csv('../datos/datos-2018.csv', index_col = 0, encoding='ISO-8859-1', sep=';')
df_2019 = pd.read_csv('../datos/datos-2019.csv', index_col = 0, encoding='ISO-8859-1', sep=';')
df_2020 = pd.read_csv('../datos/datos-2020.csv', index_col = 0, encoding='ISO-8859-1', sep=';')
df_2021 = pd.read_csv('../datos/datos-2021.csv', index_col = 0, encoding='ISO-8859-1', sep=';')

# Análisis individual de los datasets

Realizamos una inspección previa a la unión de los mismos para ver que el número de columnas y nombre no difiere.

In [ ]:
print(df_2013.shape)
print(df_2013.columns)
df_2013.head(2)

In [ ]:
print(df_2014.shape)
print(df_2014.columns)
df_2014.head(2)

In [ ]:
print(df_2015.shape)
print(df_2015.columns)
df_2015.head(2)

In [ ]:
print(df_2016.shape)
print(df_2016.columns)
df_2016.head(2)

In [ ]:
print(df_2017.shape)
print(df_2017.columns)
df_2017.head(2)

In [ ]:
print(df_2018.shape)
print(df_2018.columns)
df_2018.head(2)

In [ ]:
print(df_2019.shape)
print(df_2019.columns)
df_2019.head(2)

In [ ]:
print(df_2020.shape)
print(df_2020.columns)
df_2020.head(2)

In [ ]:
print(df_2021.shape)
print(df_2021.columns)
df_2021.head(2)

Juntamos todos los Dataframes y comprobamos su forma y tamaño final

In [ ]:
df_final = pd.concat([df_2013,df_2014,df_2015,df_2016,df_2017,df_2018,df_2019,df_2020,df_2021], ignore_index=True)
print(df_final.shape)
print(df_final.columns)

In [ ]:
df_final.head(5)

In [ ]:
df_final.isna().sum()

Revisar si hay valores duplicados, o nulos. Únicamente se observan 578 valores nulos en la columna "Data Lançamento", por lo que no son relevantes y se decide mantenerlos. 

In [ ]:
df_final.dtypes

In [ ]:
df_final["DATA LANÇAMENTO"] = pd.to_datetime(df_final["DATA LANÇAMENTO"])

Reemplazar "comas" por "puntos" y pasar las columnas "VALOR PREVISTO ATUALIZADO" ,"VALOR LANÇADO", "VALORREALIZADO",  "PERCENTUAL REALIZADO" a numérico

In [745]:
df_final['VALOR PREVISTO ATUALIZADO'] = df_final['VALOR PREVISTO ATUALIZADO'].str.replace(',', '.', regex=True) 
df_final['VALOR PREVISTO ATUALIZADO'] = pd.to_numeric(df_final['VALOR PREVISTO ATUALIZADO'])


In [746]:
df_final['VALOR LANÇADO'] = df_final['VALOR LANÇADO'].str.replace(',', '.', regex=True) 
df_final['VALOR LANÇADO'] = pd.to_numeric(df_final['VALOR LANÇADO'])

In [747]:
df_final['VALOR REALIZADO'] = df_final['VALOR REALIZADO'].str.replace(',', '.', regex=True)
df_final['VALOR REALIZADO'] = pd.to_numeric(df_final['VALOR REALIZADO'])

In [748]:
df_final['PERCENTUAL REALIZADO'] = df_final['PERCENTUAL REALIZADO'].str.replace(',', '.', regex=True)
df_final['PERCENTUAL REALIZADO'] = pd.to_numeric(df_final['PERCENTUAL REALIZADO'])

In [ ]:
df_final.head(10)

In [ ]:
df_final.describe().round(2)

In [ ]:
df_final.duplicated().sum()

In [ ]:
df_final["DATA LANÇAMENTO"]

In [ ]:
df_final["ANO EXERCÍCIO"].value_counts()

# Distribución de Ingresos por Categoría económica

In [ ]:
df_final["CATEGORIA ECONÔMICA"].value_counts()

In [ ]:
categorias_ingresos = df_final.groupby('CATEGORIA ECONÔMICA')['VALOR REALIZADO'].sum().reset_index()
print(categorias_ingresos)

Al analizar las categorías de ingresos más significativas, y su participación en los ingresos totales, destacamos que las que mayor ingresos tienen son "Receitas Correntes" y "Receitas de Capital".

In [ ]:
categorias_ingresos = df_final.groupby('CATEGORIA ECONÔMICA')['VALOR REALIZADO'].sum().reset_index()
sns.barplot(x='VALOR REALIZADO', y='CATEGORIA ECONÔMICA', data=categorias_ingresos, palette='viridis')


Podemos observar lo que se mostraba anteriormente de forma numérica, incluyendolo en forma de gráfico, siendo más visual el resultado.

In [ ]:
df_final["VALORES FINALES"] = df_final['VALOR LANÇADO'] - df_final['VALOR REALIZADO']
categorias_ingresos_lanzado = df_final.groupby('CATEGORIA ECONÔMICA')['VALORES FINALES'].mean().reset_index()
categorias_ingresos_lanzado


Calculando la diferencia promedio entre los ingresos previstos y los realizados por cada categoria, se observa que en todas las categorias, menos en la de "Sem informaçao" , el resultado es negativo. Teniendo eso en cuenta, y pensando que valor lanzado se refiere a valor declarado oficialmente por Brasil, hay parte del dinero recaudado que se está desviando o defraudando por parte de la Administración ya que no se está declarando como recibido. 

In [ ]:
a=df_final["VALOR LANÇADO"].sum()
b=df_final['VALOR REALIZADO'].sum()
c=a-b
print("El valor de lanzado es ", a)
print("El valor de realizado es ", b)
print("La diferencia es ",c)

# Análisis temporal

In [ ]:
ingresos_año = df_final.groupby("ANO EXERCÍCIO")["VALOR REALIZADO"].sum()
ingresos_año

Al analizar como ha evolucionado el valor recaudado a lo largo de los años, se observa que éste es creciente conforme pasa el tiempo, siendo el valor menos recaudado en 2013 y el recaudado más alto en 2021. 

In [ ]:
ingresos_diarios = df_final.groupby("DATA LANÇAMENTO")["VALOR REALIZADO"].sum()
ingresos_diarios

# Identificación de discrepancias

In [ ]:
df_final["DISCREPANCIAS"] = df_final["VALOR REALIZADO"] - df_final["VALOR PREVISTO ATUALIZADO"]
df_final["DISCREPANCIAS"]

Almacenamos en nuevo df, la suma total de las discrepancias de cada categoría. 

In [ ]:
df_discrepancias_por_categoria = df_final.groupby ("CATEGORIA ECONÔMICA")["DISCREPANCIAS"].sum().reset_index()
df_discrepancias_por_categoria

Así podemos observar, que las categorías "Receitas Correntes" y "Receitas de Capital" tienen una subejecución, es decir, el valor realizado es menor que el previsto.
Y por el contrario, las otras tres categorías, presentan una sobreejecución, por lo que el valor realizado es mayor que el previsto.

In [ ]:
df_final["DIFERENCIA_CONTABILIZADO"] = df_final["VALOR LANÇADO"] - df_final["VALOR PREVISTO ATUALIZADO"]
df_final["DIFERENCIA_CONTABILIZADO"]

In [ ]:
df_diferencias_contabilizadas_por_categoria = df_final.groupby ("CATEGORIA ECONÔMICA")["DIFERENCIA_CONTABILIZADO"].sum().reset_index()
df_diferencias_contabilizadas_por_categoria

# GRÁFICOS

In [ ]:
df_comparativa = df_final.groupby('CATEGORIA ECONÔMICA')[['VALOR REALIZADO','VALOR LANÇADO','VALOR PREVISTO ATUALIZADO']].sum().reset_index()
print(df_comparativa)

Gráfico donde se observa la comparación entre valores realizados por cada categoría.

In [ ]:
sns.barplot(x="CATEGORIA ECONÔMICA", y= "VALOR REALIZADO", data= df_comparativa , palette= "viridis")
plt.xticks(rotation=90)

Gráfico donde se observa la comparación entre valores previstos por cada categoría.

In [ ]:
sns.barplot(x="CATEGORIA ECONÔMICA", y= "VALOR PREVISTO ATUALIZADO", data= df_comparativa , palette= "viridis")
plt.xticks(rotation=90)

Gráfico donde se observa la comparación entre valores lanzados por cada categoría.

In [ ]:
sns.barplot(x="CATEGORIA ECONÔMICA", y= "VALOR LANÇADO", data= df_comparativa , palette= "viridis")
plt.xticks(rotation=90)

Gráfico para visualizar la evolución temporal de los ingresos realizados y previstos.

In [ ]:
df_ingresos_año = df_final.groupby("ANO EXERCÍCIO")[["VALOR REALIZADO", "VALOR PREVISTO ATUALIZADO"]].sum()
df_ingresos_año

In [ ]:
sns.barplot(x="ANO EXERCÍCIO", y= "VALOR REALIZADO", data= df_ingresos_año , palette= "viridis")

In [ ]:
sns.barplot(x="ANO EXERCÍCIO", y= "VALOR PREVISTO ATUALIZADO", data= df_ingresos_año , palette= "viridis")

Diagrama de cajas para evaluar la dispersión entre los valores previstos y realizados en las diferentes categorías.

In [ ]:
sns.boxplot(x="CATEGORIA ECONÔMICA", y= "DISCREPANCIAS" , data= df_discrepancias_por_categoria , palette= "viridis")
plt.xticks(rotation=90)

Este gráfico es poco visual, pues la dispersión entre los valores es muy elevada. 

In [ ]:
#porcentaje_nulos(df_final)